In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
from pathlib import Path
import os
import pandas as pd
import scanpy as sc
import cellink as cl
from cellink.tl._burden_testing import *
from scipy.stats import beta
import pickle
import numpy as np

import warnings
warnings.filterwarnings("ignore")

/data/nasif12/home_if12/l_mai/.conda/envs/mm_scgenetics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Mai's test notebook

In [3]:
# test only with chr22 data 
base_data_dir = Path("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/")
scdata_path = base_data_dir / "input_data/chr22_OneK1K_cohort_gene_expression_matrix_14_celltypes_w_gene_locations.h5ad.gz"
gdata_dir = "/data/ceph/hdd/project/node_09/sys_gen_students/2024_2025/project04_rare_variant_sc/input_data/filter_vcf_r08/"
DNA_LM_upstream = base_data_dir/ "input_data/annotations/onek1k_inf_scores_upstream_model.tsv"
DNA_LM_downstream = base_data_dir/ "input_data/annotations/onek1k_inf_scores_downstream_model.tsv"
vep_scores = base_data_dir/ "input_data/annotations/onek1k1_chr22_variants_annotated_vep.txt"

zarr_file = os.path.join(gdata_dir, f"chr22.dose.filtered.R2_0.8.vcz")
#eigenvec = pd.read_csv(base_data_dir / "input_data/pcdir/wgs.dose.filtered.R2_0.8.filtered.pruned.eigenvec", sep = ' ')
scdata = sc.read_h5ad(scdata_path)
gdata = cl.io.read_sgkit_zarr(zarr_file)

In [4]:
cl.tl.add_vep_annos_to_gdata(vep_scores, gdata,
                             cols_to_explode=["Consequence"],
                             cols_to_dummy=["Consequence"])

[2025-01-19 18:27:20,439] INFO:cellink.tl._annotate_snps_genotype_data: renaming id column #Uploaded_variation into snp_id
[2025-01-19 18:27:20,606] INFO:cellink.tl._annotate_snps_genotype_data: Subsetting annotations to variants that are in gdata
[2025-01-19 18:27:20,606] INFO:cellink.tl._annotate_snps_genotype_data: 0 with missing annotation
[2025-01-19 18:27:21,194] INFO:cellink.tl._annotate_snps_genotype_data: Index(['snp_id', 'Location', 'Allele', 'Gene', 'Feature', 'Feature_type',
       'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position',
       'Amino_acids', 'Codons', 'Existing_variation', 'IMPACT', 'DISTANCE',
       'STRAND', 'FLAGS', 'BIOTYPE', 'CANONICAL', 'ENSP', 'SIFT', 'PolyPhen',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'CADD_PHRED',
       'CADD_RAW', 'TSSDistance'],
      dtype='object

AnnData object with n_obs × n_vars = 1034 × 143083
    var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2', 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'
    varm: 'annotations_0', 'annotations_1', 'annotations_2', 'annotations_3', 'annotations_4', 'annotations_5', 'annotations_6', 'annotations_7'

In [5]:
def add_snp_id(DNA_LM):
    DNA_LM['snp_id'] = DNA_LM['Chromosome'] + "_" + DNA_LM['pos'].astype(str) + "_" + DNA_LM['ref'] + "_" + DNA_LM['alt']
    DNA_LM['snp_id'] = DNA_LM['snp_id'].str.replace('chr', '')
    
    # Set 'snap_id' as the index
    DNA_LM.set_index('snp_id', inplace=True)
    return DNA_LM

def reverse_and_update_snp_ids(gdata_df, dna_df):
    updated_index = []
    
    for snp_id in dna_df.index:
        if snp_id in gdata_df.index:
            updated_index.append(snp_id)
        else:
            chrom, pos, ref, alt = snp_id.split("_")
            reversed_snp_id = f"{chrom}_{pos}_{alt}_{ref}"  # Reverse ref and alt
            
            # Check if reversed_snp_id exists in data_df
            if reversed_snp_id in gdata_df.index:
                #print(f"Reversing {snp_id} to {reversed_snp_id}")
                updated_index.append(reversed_snp_id)
            else:
                print(f"Error, unknown snp_id {snp_id}")
                updated_index.append(snp_id)
    
    # Update DNA_LM's index
    dna_df.index = updated_index
    print("\nUpdated DNA_LM index:")
    print(dna_df.index)
    return dna_df

def add_maf_annotation(gdata):
    weighted_snp_maf = beta.pdf(gdata.var["maf"], 1, 25)
    gdata.varm["annotations_0"]["MAF_beta_1.25"] = weighted_snp_maf
    return gdata


def add_DNA_LM(gdata, file, chromosome, colname):

    DNA_LM = pd.read_csv(file,
                sep = '\t')
    DNA_LM = add_snp_id(DNA_LM)
    DNA_LM = reverse_and_update_snp_ids(gdata.varm["annotations_0"],
                                        DNA_LM[DNA_LM["Chromosome"]==f"chr{chromosome}"])

    gdata.varm["annotations_0"][colname] = DNA_LM["influence_score"].reindex(gdata.varm["annotations_0"].index)

    # Rename the merged column if needed
    gdata.varm["annotations_0"].rename(columns={"influence_score": colname}, inplace=True)
    return gdata

In [6]:
# add maf annotaion to gdata
print(f"add maf annotation to gdata ")
gdata = add_maf_annotation(gdata)

# add DNA_LM annotations (downstream and upstream models) to gdata 
print(f"add DNA_LM annotation to gdata ")
gdata = add_DNA_LM(gdata, file=DNA_LM_upstream, chromosome="22", colname='DNA_LM_up')
gdata = add_DNA_LM(gdata, file=DNA_LM_downstream, chromosome="22", colname='DNA_LM_down')

add maf annotation to gdata 
add DNA_LM annotation to gdata 

Updated DNA_LM index:
Index(['22_17069391_G_A', '22_17093288_C_T', '22_17102725_A_G',
       '22_17115621_T_A', '22_17117130_G_A', '22_17128890_A_G',
       '22_17202942_T_G', '22_17227050_G_T', '22_17274178_A_T',
       '22_17276558_C_T',
       ...
       '22_51171783_C_T', '22_51172439_G_A', '22_51172880_G_A',
       '22_51175039_A_C', '22_51175382_A_G', '22_51175798_C_T',
       '22_51177291_C_T', '22_51180520_C_T', '22_51180534_A_G',
       '22_51185359_G_A'],
      dtype='object', length=46826)

Updated DNA_LM index:
Index(['22_17069391_G_A', '22_17093288_C_T', '22_17102725_A_G',
       '22_17115621_T_A', '22_17117130_G_A', '22_17128890_A_G',
       '22_17202942_T_G', '22_17227050_G_T', '22_17274178_A_T',
       '22_17276558_C_T',
       ...
       '22_51171783_C_T', '22_51172439_G_A', '22_51172880_G_A',
       '22_51175039_A_C', '22_51175382_A_G', '22_51175798_C_T',
       '22_51177291_C_T', '22_51180520_C_T', '22_511

## Check results

In [8]:
test_res = pd.read_pickle(base_data_dir/"output/chr22_computed_burdens_test_10_genes.pkl")
test_res

CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  DNA_LM_mixed  \
id                                                                           
1_1          9350.801  13.224558    22.916096   43612.917035     13.224558   
2_2         10775.218  16.520484    28.387657   52127.930292     16.520484   
3_3          8327.426  12.240323    21.604692   39069.213569     12.240323   
4_4          9439.675  18.346526    32.699742   44983.557785     18.346526   
6_6          9054.896  13.340017    23.019223   42525.702648     13.340017   
...               ...        ...          ...            ...           ...   
1077_1078    9227.155  15.636985    27.382034   42677.295555     15.636985   
1078_1079    9411.798  14.626046    25.363997   44142.005875     14.626046   
1079_1080    9862.022  13.470969    23.919521   44060.525919     13.470969   
1080_1081    8392.390  13.858525    24.546299   40866.836483     13.858525   
1081_1082    9015.698  15.057642    26.508232   42943.342890     15.057642   

                    Geneid  
id                          
1_1        ENSG00000233866  
2_2        ENSG00000233866  
3_3        ENSG00000233866  
4_4        ENSG00000233866  
6_6        ENSG00000233866  
...                    ...  
1077_1078  ENSG00000267338  
1078_1079  ENSG00000267338  
1079_1080  ENSG00000267338  
1080_1081  ENSG00000267338  
1081_1082  ENSG00000267338  

[9810 rows x 6 columns]

## Save Onek1k chr 22 with gene location for test purposes

In [8]:
scdata_full_path = base_data_dir / "input_data/OneK1K_cohort_gene_expression_matrix_14_celltypes_w_gene_locations.h5ad.gz"
scdata = sc.read_h5ad(scdata_full_path)
scdata

AnnData object with n_obs × n_vars = 1272489 × 32738
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'pool', 'individual', 'percent.mt', 'latent', 'nCount_SCT', 'nFeature_SCT', 'cell_type', 'cell_label', 'sex', 'age'
    var: 'GeneSymbol', 'features', 'chromosome', 'start', 'end'

In [5]:
scdata_22 = scdata[:, scdata.var["chromosome"] == "22"]
scdata_22

View of AnnData object with n_obs × n_vars = 1272489 × 655
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'pool', 'individual', 'percent.mt', 'latent', 'nCount_SCT', 'nFeature_SCT', 'cell_type', 'cell_label', 'sex', 'age'
    var: 'GeneSymbol', 'features', 'chromosome', 'start', 'end'

In [6]:
scdata_22.var

GeneSymbol        features chromosome       start  \
Geneid                                                                   
ENSG00000233866     LA16c-4G1.3     LA16c-4G1.3         22  15915800.0   
ENSG00000225255   LA16c-83F12.6   LA16c-83F12.6         22  15741297.0   
ENSG00000198062           POTEH           POTEH         22  15690026.0   
ENSG00000236666       POTEH-AS1       POTEH-AS1         22  15703403.0   
ENSG00000230471     LA16c-2F2.8     LA16c-2F2.8         22  15625205.0   
...                         ...             ...        ...         ...   
ENSG00000251322          SHANK3          SHANK3         22  50674408.0   
ENSG00000225929      AC000036.4      AC000036.4         22  50738173.0   
ENSG00000100312             ACR             ACR         22  50738196.0   
ENSG00000254499      AC002056.5      AC002056.5         22  50743520.0   
ENSG00000079974          RABL2B          RABL2B         22  50783667.0   

                        end  
Geneid                       
ENSG00000233866  15914721.0  
ENSG00000225255  15779680.0  
ENSG00000198062  15721631.0  
ENSG00000236666  15699361.0  
ENSG00000230471  15557577.0  
...                     ...  
ENSG00000251322  50733212.0  
ENSG00000225929  50735813.0  
ENSG00000100312  50745339.0  
ENSG00000254499  50740593.0  
ENSG00000079974  50767501.0  

[655 rows x 5 columns]

In [7]:
scdata_22.write(base_data_dir / "input_data/chr22_OneK1K_cohort_gene_expression_matrix_14_celltypes_w_gene_locations.h5ad.gz", compression="gzip")

## Test distance calculations

In [7]:
#preprocess scdata
sc.pp.normalize_total(scdata, target_sum=1e4)  # Normalize total counts per cell
sc.pp.log1p(scdata)  # Apply log-transform

In [9]:
gdata.varm["annotations_0"]

Consequence_start_lost PHENO  gnomADe_AF Feature_type  \
snp_id                                                                   
22_16849573_A_G                       0     -         NaN            -   
22_16849971_A_T                       0     -         NaN            -   
22_16850437_G_A                       0     -         NaN            -   
22_16851225_C_T                       0     -         NaN            -   
22_16851356_C_T                       0     -         NaN            -   
...                                 ...   ...         ...          ...   
22_51202748_A_G                       0     -         NaN   Transcript   
22_51208568_G_T                       0     -         NaN   Transcript   
22_51211031_A_G                       0     -         NaN   Transcript   
22_51213613_C_T                       0     -         NaN   Transcript   
22_51216564_T_C                       0     -         NaN   Transcript   

                CLIN_SIG  gnomADe_OTH_AF  Consequence_intergenic_variant  \
snp_id                                                                     
22_16849573_A_G        -             NaN                               1   
22_16849971_A_T        -             NaN                               1   
22_16850437_G_A        -             NaN                               1   
22_16851225_C_T        -             NaN                               1   
22_16851356_C_T        -             NaN                               1   
...                  ...             ...                             ...   
22_51202748_A_G        -             NaN                               0   
22_51208568_G_T        -             NaN                               0   
22_51211031_A_G        -             NaN                               0   
22_51213613_C_T        -             NaN                               0   
22_51216564_T_C        -             NaN                               0   

                    Location  gnomADe_AFR_AF Allele  ...  \
snp_id                                               ...   
22_16849573_A_G  22:16849573             NaN      G  ...   
22_16849971_A_T  22:16849971             NaN      T  ...   
22_16850437_G_A  22:16850437             NaN      A  ...   
22_16851225_C_T  22:16851225             NaN      T  ...   
22_16851356_C_T  22:16851356             NaN      T  ...   
...                      ...             ...    ...  ...   
22_51202748_A_G  22:51202748             NaN      G  ...   
22_51208568_G_T  22:51208568             NaN      T  ...   
22_51211031_A_G  22:51211031             NaN      G  ...   
22_51213613_C_T  22:51213613             NaN      T  ...   
22_51216564_T_C  22:51216564             NaN      C  ...   

                 Consequence_splice_acceptor_variant  \
snp_id                                                 
22_16849573_A_G                                    0   
22_16849971_A_T                                    0   
22_16850437_G_A                                    0   
22_16851225_C_T                                    0   
22_16851356_C_T                                    0   
...                                              ...   
22_51202748_A_G                                    0   
22_51208568_G_T                                    0   
22_51211031_A_G                                    0   
22_51213613_C_T                                    0   
22_51216564_T_C                                    0   

                 Consequence_splice_donor_5th_base_variant  \
snp_id                                                       
22_16849573_A_G                                          0   
22_16849971_A_T                                          0   
22_16850437_G_A                                          0   
22_16851225_C_T                                          0   
22_16851356_C_T                                          0   
...                                                    ...   
22_51202748_A_G                                          0 

In [10]:
gdata_anno0 = gdata.varm["annotations_0"]

In [8]:
def _get_burden(gd_gene, weight_col):
    this_weights = np.array(gd_gene.varm['annotations_0'][weight_col])
    g_weigthed = gd_gene.X * this_weights
    this_burdens = np.nansum(g_weigthed, axis = 1) #TODO implement alternative weighting functions
    return this_burdens

def _calc_tss_distance_per_gene(variants_df,
                                gene_start,
                                gene_end,
                                col_name_distance,
                                col_name_saige=""):
    """
    Calculate absolute TSS Distance.

    Parameters:
        variants_df (pd.DataFrame): DataFrame with SNP ID as index and contains variant position in column "Position"
        gene_start (int): Start of Gene aka TS
        col_name_distance (str): name of TSS column
        col_name_saige (str, optional): determines whether saige formula should be calculated as well.

    Returns:
        distances_df (pd.DataFrame): Dataframe with tss distance per variant
    """
    distances = {"snp_id": [], col_name_distance: []}
    saige_distances = []
    #print(variants_df.index)
    
    if gene_start > gene_end:
        gene_start, gene_end = gene_end, gene_start  # Reverse strand handling

    for i, row in variants_df.iterrows():
        if gene_start <= row["Position"] <= gene_end:
            distances["snp_id"].append(i)
            distances[col_name_distance].append(0)  # Variant is within the gene

        # Calculate absolute distances to start and end positions
        distance_to_start = abs(row["Position"] - gene_start)  # upstream
        distance_to_end = abs(row["Position"] - gene_end)  # downstream

        # Get the minimum distance = correct distance
        distance = min(distance_to_start, distance_to_end)
        distances["snp_id"].append(i)
        distances[col_name_distance].append(distance)

        # calculate saige if parameter is set
        if col_name_saige != "":
            distance_saige = np.exp(-1e-5 * distance)
            saige_distances.append(distance_saige)

    # add saige to final dataframe if parameter is set
    if col_name_saige != "":
        distances[col_name_saige] = saige_distances

    # return dataframe with tss distance per variant
    distance_df = pd.DataFrame(distances)
    distance_df.set_index("snp_id", inplace=True)
    #print(distance_df)
    return distance_df


def _find_snps_near_gene(gdata,
                         gene_chrom,
                         gene_start,
                         gene_end,
                         bp_range=10000):
    """
    Finds SNPs within a specified range of a gene's location.

    Parameters:
        gdata (pd.DataFrame): DataFrame with a 'Location' column in the format "chromosome:position".
        gene_chrom, gene_start, gene_end (int)
        bp_range (int): Range in base pairs to search upstream and downstream.

    Returns:
        pd.DataFrame: With data of SNPs within the specified range.
    """
    # Parse the gene location
    #import ipdb; ipdb.set_trace()

    # Extract chromosome and position from the SNPs
    gene_chrom = str(gene_chrom)
    gdata_df = gdata.copy()
    gdata_df[['Chromosome', 'Position']] = gdata_df['Location'].str.split(':', expand=True)
    gdata_df['Position'] = gdata_df['Position'].astype(int)

    # Filter for SNPs within the range
    # snps_in_range = gdata_df[
    #     (gdata_df['Chromosome'] == gene_chrom) &
    #     (gdata_df['Position'] >= gene_start - bp_range) &
    #     (gdata_df['Position'] <= gene_end + bp_range)
    # ]

    if gene_start < gene_end:
        # forward strand
        snps_upstream = gdata_df[
            (gdata_df['Chromosome'] == gene_chrom) &
            (gdata_df['Position'] >= gene_start - bp_range)
        ]
        snps_downstream = gdata_df[
            (gdata_df['Chromosome'] == gene_chrom) &
            (gdata_df['Position'] <= gene_start + bp_range)
        ]
    else:
        # reverse strand
        snps_upstream = gdata_df[
            (gdata_df['Chromosome'] == gene_chrom) &
            (gdata_df['Position'] >= gene_start + bp_range)
        ]
        snps_downstream = gdata_df[
            (gdata_df['Chromosome'] == gene_chrom) &
            (gdata_df['Position'] <= gene_start - bp_range)
        ]

    # return snps_in_range.index
    return snps_upstream, snps_downstream


def _compute_burdens_for_gene(this_gd,
                              this_gene,
                              gene_chrom,
                              gene_start,
                              gene_end,
                              weight_cols,
                              annotation_varm="annotations_0",
                              window_size=100000,
                              DNA_LM_up="",
                              DNA_LM_down="",
                              DNA_LM_mixed="DNA_LM_mixed",
                              GENE_TSS_DISTANCE="",
                              GENE_TSS_DISTANCE_SAIGE=""):
    """
    Compute burdenscores for a given gene and given annotations

    Parameters:
        this_gd (pd.DataFrame): ddata.gdata.
        this_gene (str): Ensemble ID.
        gene_chrom, gene_start, gene_end (int)
        weight_cols (list): colnames of variant annotations to compute burden scores for.
        annotation_varm (str): key for pd.DataFrame (gdata.varm[key])
        window_size (int)
        DNA_LM_up (str): colname for DNA_LM upstream model
                    if empty, mixed model is not computed
        DNA_LM_down (str): colname for DNA_LM downstream model
                    if empty, mixed model is not computed
        DNA_LM_mixed (str): name of mixed model column
        GENE_TSS_DISTANCE (str): name for tss distance column
        GENE_TSS_DISTANCE_SAIGE (str): name for tss distance saige column

    Returns:
        pd.DataFrame containing burden scores for this_gene across the weight_cols
    """
    # Return a dataframe with None entries for weight cols, if gene location is nan (this means it could not be found in ensembl)
    if np.isnan(gene_chrom):
        print(f"Failed to retrieve location for gene {this_gene}. No Burden scores computed.")
        # Create a DataFrame with None for all the weight columns
        empty_burdens = pd.DataFrame(
            None,
            index=this_gd.obs.index,  # Assuming these are the sample indices
            columns=weight_cols
        )
        # Add the Geneid column
        empty_burdens["Geneid"] = this_gene
        return empty_burdens

    # Filter the variants using the SNP location and gene location
    this_vars_up_df, this_vars_down_df = _find_snps_near_gene(this_gd.varm[annotation_varm], gene_chrom, gene_start, gene_end, bp_range=window_size)

    # get snps IDs
    this_vars_down = this_vars_down_df.index
    this_vars_up = this_vars_up_df.index

    gd_gene = this_gd[:, this_vars_up.append(this_vars_down).unique()].copy()

    # if mixed model is computed, add column for DNA_LM mixed model
    if DNA_LM_up != "":
        # Add the "DNA_LM_mixed" column to the annotations_0 DataFrame
        gd_gene.varm["annotations_0"][DNA_LM_mixed] = np.nan  # Initialize the column with NaN

        # Assign values to the "DNA_LM_mixed" column based on the conditions
        gd_gene.varm["annotations_0"].loc[this_vars_up, DNA_LM_mixed] = gd_gene.varm["annotations_0"].loc[this_vars_up, DNA_LM_up]
        gd_gene.varm["annotations_0"].loc[this_vars_down, DNA_LM_mixed] = gd_gene.varm["annotations_0"].loc[this_vars_down, DNA_LM_down]

    if GENE_TSS_DISTANCE_SAIGE != "": # calc GENE_TSS_DISTANCE and GENE_TSS_DISTANCE_SAIGE
        gd_gene.varm["annotations_0"][GENE_TSS_DISTANCE] = np.nan # Initialize the column with NaN
        gd_gene.varm["annotations_0"][GENE_TSS_DISTANCE_SAIGE] = np.nan  # Initialize the column with NaN
        # calculate GENE_TSS_DISTANCE and GENE_TSS_DISTANCE_SAIGE independent of up or downstream
        all_variants = pd.concat([this_vars_up_df, this_vars_down_df], axis=0)
        distances = _calc_tss_distance_per_gene(all_variants, gene_start, gene_end, GENE_TSS_DISTANCE, GENE_TSS_DISTANCE_SAIGE)
        #print(distances)
        #print(distances.index)
        # add GENE_TSS_DISTANCE and GENE_TSS_DISTANCE_SAIGE to annotation 0
        gd_gene.varm["annotations_0"].loc[distances.index, GENE_TSS_DISTANCE] = distances[GENE_TSS_DISTANCE]
        gd_gene.varm["annotations_0"].loc[distances.index, GENE_TSS_DISTANCE_SAIGE] = distances[GENE_TSS_DISTANCE_SAIGE]
    elif GENE_TSS_DISTANCE != "":  # calc only GENE_TSS_DISTANCE
        gd_gene.varm["annotations_0"][GENE_TSS_DISTANCE] = np.nan  # Initialize the column with NaN
        # calculate tss distance independent of up or downstream
        all_variants = pd.concat([this_vars_up_df, this_vars_down_df])
        tss_distances = _calc_tss_distance_per_gene(all_variants, gene_start, gene_end, GENE_TSS_DISTANCE)
        # add to gd_gene
        gd_gene.varm["annotations_0"].loc[tss_distances.index, GENE_TSS_DISTANCE] = tss_distances[GENE_TSS_DISTANCE]

    all_burdens_this_gene = []
    for weight_col in weight_cols:
        this_burden = _get_burden(gd_gene, weight_col)
        all_burdens_this_gene.append(this_burden)

    all_burdens_this_gene = np.stack(all_burdens_this_gene, axis=1)
    all_burdens_this_gene = pd.DataFrame(all_burdens_this_gene, index=gd_gene.obs.index, columns=weight_cols)
    all_burdens_this_gene["Geneid"] = this_gene

    return all_burdens_this_gene


In [59]:
this_gd = gdata[:, gdata.var["maf"] < 0.05]

In [68]:
_compute_burdens_for_gene(this_gd, "ENSG00000233866", 22, 15915800.0, 15914721.0, ["GENE_TSS_DISTANCE","GENE_TSS_DISTANCE_SAIGE"], "annotations_0", 10000,GENE_TSS_DISTANCE="GENE_TSS_DISTANCE", GENE_TSS_DISTANCE_SAIGE="GENE_TSS_DISTANCE_SAIGE")

GENE_TSS_DISTANCE  GENE_TSS_DISTANCE_SAIGE           Geneid
id                                                                     
1_1              5.974337e+10                 0.000005  ENSG00000233866
2_2              6.255511e+10                 0.000030  ENSG00000233866
3_3              5.302204e+10                 0.000003  ENSG00000233866
4_4              6.431704e+10                 0.000004  ENSG00000233866
6_6              5.468808e+10                 0.000004  ENSG00000233866
...                       ...                      ...              ...
1096_845_2       5.767224e+10                 0.000003  ENSG00000233866
1100_914_2       5.312065e+10                 0.000004  ENSG00000233866
1102_932_2       6.263424e+10                 0.000003  ENSG00000233866
1103_926_2       6.648737e+10                 0.000004  ENSG00000233866
1104_920_2       5.762447e+10                 0.000003  ENSG00000233866

[1034 rows x 3 columns]

In [70]:
_compute_burdens_for_gene(this_gd, "ENSG00000233866", 22, 15915800.0, 15914721.0, ["GENE_TSS_DISTANCE"], "annotations_0", 10000,GENE_TSS_DISTANCE="GENE_TSS_DISTANCE")

GENE_TSS_DISTANCE           Geneid
id                                            
1_1              5.974337e+10  ENSG00000233866
2_2              6.255511e+10  ENSG00000233866
3_3              5.302204e+10  ENSG00000233866
4_4              6.431704e+10  ENSG00000233866
6_6              5.468808e+10  ENSG00000233866
...                       ...              ...
1096_845_2       5.767224e+10  ENSG00000233866
1100_914_2       5.312065e+10  ENSG00000233866
1102_932_2       6.263424e+10  ENSG00000233866
1103_926_2       6.648737e+10  ENSG00000233866
1104_920_2       5.762447e+10  ENSG00000233866

[1034 rows x 2 columns]

In [71]:
_compute_burdens_for_gene(this_gd, "ENSG00000233866", 22, 15915800.0, 15914721.0, ["GENE_TSS_DISTANCE_SAIGE"], "annotations_0", 10000,GENE_TSS_DISTANCE_SAIGE="GENE_TSS_DISTANCE_SAIGE")

GENE_TSS_DISTANCE_SAIGE           Geneid
id                                                  
1_1                        0.000005  ENSG00000233866
2_2                        0.000030  ENSG00000233866
3_3                        0.000003  ENSG00000233866
4_4                        0.000004  ENSG00000233866
6_6                        0.000004  ENSG00000233866
...                             ...              ...
1096_845_2                 0.000003  ENSG00000233866
1100_914_2                 0.000004  ENSG00000233866
1102_932_2                 0.000003  ENSG00000233866
1103_926_2                 0.000004  ENSG00000233866
1104_920_2                 0.000003  ENSG00000233866

[1034 rows x 2 columns]

In [21]:
def compute_burdens(ddata,
                    max_af=0.05,
                    weight_cols=["DISTANCE", "CADD_PHRED"],
                    annotations_varm="annotations_0",
                    window_size=100000,
                    DNA_LM_up="",
                    DNA_LM_down="",
                    DNA_LM_mixed="DNA_LM_mixed",
                    GENE_TSS_DISTANCE="",
                    GENE_TSS_DISTANCE_SAIGE=""):
    """Compute gene burdens for each gene and sample using different variant annotations

    Parameters
    ----------
    ddata : ddata
        _description_
    max_af : float, optional
        maximum variant minor allele frequency, by default 0.05
    weight_cols : list, optional
        variant annotations used for weighting (columns of gdata.varm.annotations_0), by default ["DISTANCE", "CADD_PHRED"]
    annotations_varm: str, optional
        key for gdata.varm dataframe (eg: gdata.varm["annotations_0"])
    window_size: int, optional
        range around gene TSS, in which variants are regarded for gene burden scores
    DNA_LM_up: str, optional, if DNA_LM mixed model should be computed
        colname of DNA_LM score upstream model
    DNA_LM_down: str, optional, if DNA_LM mixed model should be computed
        colname of DNA_LM score downstream model
    DNA_LM_mixed: str, optional
        colname of DNA_LM score mixed model
    GENE_TSS_DISTANCE: str, optional
        colname for TSS distances and also flag to compute its burden 
    GENE_TSS_DISTANCE_SAIGE:str, optional
        colname for TSS distances using saige formula and also flag to compute its burden 
    Returns
    -------
    pandas.DataFrame
        gene burdens for all genes, individuals and all annotations in weightcols
    """
    if (DNA_LM_up != "" and DNA_LM_down == "") or (DNA_LM_up == "" and DNA_LM_down != ""):
        raise ValueError("If you want to compute the burden scores using the DNA_LM mixed model, you must set both DNA_LM_up and DNA_LM_down, else leave both empty.")

    this_gd = ddata.gdata.copy()
    this_ad = ddata.adata.copy()
    this_gd = this_gd[:, this_gd.var["maf"] < max_af]
    all_burdens = []

    if not all(col in this_ad.var.columns for col in ["chromosome", "start", "end"]):
        # compute all the gene locations
        this_ad.var[['chromosome', 'start', 'end']] = this_ad.var.index.to_series().apply(
            lambda x: pd.Series(_get_gene_location(x))
        )

    # add mixed model to the weight cols for which burden score is computed
    if DNA_LM_up != "" and DNA_LM_down != "" and DNA_LM_mixed not in weight_cols:
        weight_cols.append(DNA_LM_mixed)
        if DNA_LM_up not in weight_cols:
            weight_cols.append(DNA_LM_up)
        if DNA_LM_down not in weight_cols:
            weight_cols.append(DNA_LM_down)

    # add tss distance to the weight cols to initialize burden score computations
    if GENE_TSS_DISTANCE != "" and GENE_TSS_DISTANCE not in weight_cols:
        weight_cols.append(GENE_TSS_DISTANCE)

    # add tss distance saige to the weight cols to initialize burden score computations
    if GENE_TSS_DISTANCE_SAIGE != "":
        if GENE_TSS_DISTANCE_SAIGE not in weight_cols:
            weight_cols.append(GENE_TSS_DISTANCE_SAIGE)
        if GENE_TSS_DISTANCE == "":  # if saige is set then tss distance has to be calculated too
            GENE_TSS_DISTANCE = "GENE_TSS_DISTANCE"

    for gene in tqdm(this_ad.var.index[0:1]):
        gene_chrom = int(this_ad.var.loc[gene, "chromosome"])
        gene_start = int(this_ad.var.loc[gene, "start"])
        gene_end = int(this_ad.var.loc[gene, "end"])

        this_b = _compute_burdens_for_gene(this_gd, gene, gene_chrom, gene_start, gene_end, weight_cols, annotations_varm, window_size, DNA_LM_up, DNA_LM_down, DNA_LM_mixed, GENE_TSS_DISTANCE, GENE_TSS_DISTANCE_SAIGE)
        all_burdens.append(this_b)

    all_burdens = pd.concat(all_burdens)

    return all_burdens

In [10]:
data = cl.DonorData(adata=scdata, gdata=gdata, donor_key_in_sc_adata="individual")

[2025-01-19 18:33:15,519] INFO:cellink._core.donordata: Keeping 981/1034 donors
[2025-01-19 18:33:15,520] INFO:cellink._core.donordata: Dropping 53/1034 donors from genetic data
[2025-01-19 18:33:15,520] INFO:cellink._core.donordata: Dropping 0/981 donors from single-cell data


In [20]:
compute_burdens(data, max_af=0.05, weight_cols=["CADD_PHRED", "DNA_LM_up", "DNA_LM_down", "MAF_beta_1.25"], window_size=100000, DNA_LM_up="DNA_LM_up", DNA_LM_down="DNA_LM_down",GENE_TSS_DISTANCE="GENE_TSS_DISTANCE", GENE_TSS_DISTANCE_SAIGE="GENE_TSS_DISTANCE_SAIGE")

ENSG00000233866


  0%|          | 0/1 [00:00<?, ?it/s]

ENSG00000233866



00%|██████████| 1/1 [00:05<00:00,  5.22s/it]

CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  DNA_LM_mixed  \
id                                                                           
1_1          9350.801  13.224558    22.916096   43612.917035     13.224558   
2_2         10775.218  16.520484    28.387657   52127.930292     16.520484   
3_3          8327.426  12.240323    21.604692   39069.213569     12.240323   
4_4          9439.675  18.346526    32.699742   44983.557785     18.346526   
6_6          9054.896  13.340017    23.019223   42525.702648     13.340017   
...               ...        ...          ...            ...           ...   
1077_1078    9227.155  15.636985    27.382034   42677.295555     15.636985   
1078_1079    9411.798  14.626046    25.363997   44142.005875     14.626046   
1079_1080    9862.022  13.470969    23.919521   44060.525919     13.470969   
1080_1081    8392.390  13.858525    24.546299   40866.836483     13.858525   
1081_1082    9015.698  15.057642    26.508232   42943.342890     15.057642   

           GENE_TSS_DISTANCE  GENE_TSS_DISTANCE_SAIGE           Geneid  
id                                                                      
1_1             5.974337e+10                 0.000005  ENSG00000233866  
2_2             6.255511e+10                 0.000030  ENSG00000233866  
3_3             5.302204e+10                 0.000003  ENSG00000233866  
4_4             6.431704e+10                 0.000004  ENSG00000233866  
6_6             5.468808e+10                 0.000004  ENSG00000233866  
...                      ...                      ...              ...  
1077_1078       5.507287e+10                 0.000006  ENSG00000233866  
1078_1079       6.204754e+10                 0.000096  ENSG00000233866  
1079_1080       6.135349e+10                 0.000005  ENSG00000233866  
1080_1081       5.485095e+10                 0.000003  ENSG00000233866  
1081_1082       6.015882e+10                 0.000054  ENSG00000233866  

[981 rows x 8 columns]

In [22]:
compute_burdens(data, max_af=0.05, weight_cols=["CADD_PHRED", "DNA_LM_up", "DNA_LM_down", "MAF_beta_1.25"], window_size=100000, DNA_LM_up="DNA_LM_up", DNA_LM_down="DNA_LM_down",GENE_TSS_DISTANCE="GENE_TSS_DISTANCE")


00%|██████████| 1/1 [00:04<00:00,  4.84s/it]

CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  DNA_LM_mixed  \
id                                                                           
1_1          9350.801  13.224558    22.916096   43612.917035     13.224558   
2_2         10775.218  16.520484    28.387657   52127.930292     16.520484   
3_3          8327.426  12.240323    21.604692   39069.213569     12.240323   
4_4          9439.675  18.346526    32.699742   44983.557785     18.346526   
6_6          9054.896  13.340017    23.019223   42525.702648     13.340017   
...               ...        ...          ...            ...           ...   
1077_1078    9227.155  15.636985    27.382034   42677.295555     15.636985   
1078_1079    9411.798  14.626046    25.363997   44142.005875     14.626046   
1079_1080    9862.022  13.470969    23.919521   44060.525919     13.470969   
1080_1081    8392.390  13.858525    24.546299   40866.836483     13.858525   
1081_1082    9015.698  15.057642    26.508232   42943.342890     15.057642   

           GENE_TSS_DISTANCE           Geneid  
id                                             
1_1             5.974337e+10  ENSG00000233866  
2_2             6.255511e+10  ENSG00000233866  
3_3             5.302204e+10  ENSG00000233866  
4_4             6.431704e+10  ENSG00000233866  
6_6             5.468808e+10  ENSG00000233866  
...                      ...              ...  
1077_1078       5.507287e+10  ENSG00000233866  
1078_1079       6.204754e+10  ENSG00000233866  
1079_1080       6.135349e+10  ENSG00000233866  
1080_1081       5.485095e+10  ENSG00000233866  
1081_1082       6.015882e+10  ENSG00000233866  

[981 rows x 7 columns]

In [23]:
compute_burdens(data, max_af=0.05, weight_cols=["CADD_PHRED", "DNA_LM_up", "DNA_LM_down", "MAF_beta_1.25"], window_size=100000, DNA_LM_up="DNA_LM_up", DNA_LM_down="DNA_LM_down", GENE_TSS_DISTANCE_SAIGE="GENE_TSS_DISTANCE_SAIGE")


00%|██████████| 1/1 [00:04<00:00,  4.95s/it]

CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  DNA_LM_mixed  \
id                                                                           
1_1          9350.801  13.224558    22.916096   43612.917035     13.224558   
2_2         10775.218  16.520484    28.387657   52127.930292     16.520484   
3_3          8327.426  12.240323    21.604692   39069.213569     12.240323   
4_4          9439.675  18.346526    32.699742   44983.557785     18.346526   
6_6          9054.896  13.340017    23.019223   42525.702648     13.340017   
...               ...        ...          ...            ...           ...   
1077_1078    9227.155  15.636985    27.382034   42677.295555     15.636985   
1078_1079    9411.798  14.626046    25.363997   44142.005875     14.626046   
1079_1080    9862.022  13.470969    23.919521   44060.525919     13.470969   
1080_1081    8392.390  13.858525    24.546299   40866.836483     13.858525   
1081_1082    9015.698  15.057642    26.508232   42943.342890     15.057642   

           GENE_TSS_DISTANCE_SAIGE           Geneid  
id                                                   
1_1                       0.000005  ENSG00000233866  
2_2                       0.000030  ENSG00000233866  
3_3                       0.000003  ENSG00000233866  
4_4                       0.000004  ENSG00000233866  
6_6                       0.000004  ENSG00000233866  
...                            ...              ...  
1077_1078                 0.000006  ENSG00000233866  
1078_1079                 0.000096  ENSG00000233866  
1079_1080                 0.000005  ENSG00000233866  
1080_1081                 0.000003  ENSG00000233866  
1081_1082                 0.000054  ENSG00000233866  

[981 rows x 7 columns]

In [24]:
test = pd.read_csv(base_data_dir/"output/example_output/burden_CellTypeCD8-NC_Chrom22.csv")
test

burden_gene      target_gene             burden_type    pvalue  \
0     ENSG00000100181  ENSG00000100181                DISTANCE  0.541026   
1     ENSG00000100181  ENSG00000100181              CADD_PHRED  0.285429   
2     ENSG00000100181  ENSG00000100181  DNA_LM_influence_score  0.341302   
3     ENSG00000100181  ENSG00000100181           MAF_beta_1.25  0.365920   
4     ENSG00000237438  ENSG00000237438                DISTANCE  0.321480   
...               ...              ...                     ...       ...   
1867  ENSG00000100299  ENSG00000100299           MAF_beta_1.25  0.958578   
1868  ENSG00000079974  ENSG00000079974                DISTANCE  1.000000   
1869  ENSG00000079974  ENSG00000079974              CADD_PHRED  1.000000   
1870  ENSG00000079974  ENSG00000079974  DNA_LM_influence_score  1.000000   
1871  ENSG00000079974  ENSG00000079974           MAF_beta_1.25  1.000000   

          beta cell_type  chrom  
0     0.000362    CD8 NC     22  
1     0.000635    CD8 NC     22  
2     0.000566    CD8 NC     22  
3     0.000537    CD8 NC     22  
4     0.000314    CD8 NC     22  
...        ...       ...    ...  
1867 -0.000063    CD8 NC     22  
1868       NaN    CD8 NC     22  
1869       NaN    CD8 NC     22  
1870       NaN    CD8 NC     22  
1871       NaN    CD8 NC     22  

[1872 rows x 7 columns]

In [25]:
all_res = pd.read_pickle(base_data_dir/"output/all_results_DNA_LM_and_MAF.pkl")

In [26]:
all_res

burden_gene      target_gene             burden_type    pvalue  \
0   ENSG00000100181  ENSG00000100181                DISTANCE  0.766616   
1   ENSG00000100181  ENSG00000100181              CADD_PHRED  0.840065   
2   ENSG00000100181  ENSG00000100181  DNA_LM_influence_score  0.596951   
3   ENSG00000100181  ENSG00000100181           MAF_beta_1.25  0.896485   
0   ENSG00000237438  ENSG00000237438                DISTANCE  0.521166   
..              ...              ...                     ...       ...   
3   ENSG00000100288  ENSG00000100288           MAF_beta_1.25  0.093014   
0   ENSG00000079974  ENSG00000079974                DISTANCE  1.000000   
1   ENSG00000079974  ENSG00000079974              CADD_PHRED  1.000000   
2   ENSG00000079974  ENSG00000079974  DNA_LM_influence_score  1.000000   
3   ENSG00000079974  ENSG00000079974           MAF_beta_1.25  1.000000   

        beta     cell_type chrom  pvalue_corrected  significant  
0   0.000125        CD8 ET    22      19376.982274        False  
1  -0.000085        CD8 ET    22      21233.491502        False  
2  -0.000224        CD8 ET    22      15088.544000        False  
3  -0.000055        CD8 ET    22      22659.563673        False  
0   0.000112        CD8 ET    22      13172.988603        False  
..       ...           ...   ...               ...          ...  
3  -0.030436  Erythrocytes    22       2351.011086        False  
0        NaN  Erythrocytes    22      25276.000000        False  
1        NaN  Erythrocytes    22      25276.000000        False  
2        NaN  Erythrocytes    22      25276.000000        False  
3        NaN  Erythrocytes    22      25276.000000        False  

[25276 rows x 9 columns]